# DeepHyper Analytics for Neural Architecture Search - Single Study
**path to data log directory**: data_2021-07-14_01.json

for customization please see: https://matplotlib.org/api/matplotlib_configuration_api.html

## Setup & Data loading

In [1]:
path_to_logdir = '/home/bwilson/DL/deephyper'

In [2]:
import os
import json
from pprint import pprint
from datetime import datetime

import matplotlib
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, Markdown

from deephyper.core.logs.topk import output_best_configuration

In [3]:
init_infos_path = os.path.join(path_to_logdir, "init_infos.json")
results_path = os.path.join(path_to_logdir, "results.csv")
balsam_workload_path = os.path.join(path_to_logdir, "balsam-workload.json")

In [5]:
def to_sec(ts):
    try:
        return datetime.strptime(ts, '%Y-%m-%d %H:%M:%S').timestamp()
    except:
        return datetime.strptime(ts, '%Y-%m-%d %H:%M:%S.%f').timestamp()

def load_json(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

print(f"init_infos_path: {init_infos_path}")
init_infos = load_json(init_infos_path)
results_df = pd.read_csv(results_path)
try:
    balsam_workload = load_json(balsam_workload_path)
except FileNotFoundError:
    print("Balsam workload not found...")

start_timestamp = init_infos["start_timestamp"]
print(list(init_infos.keys()))

init_infos_path: ./init_infos.json


FileNotFoundError: [Errno 2] No such file or directory: './init_infos.json'

### Matplotlib style

In [ ]:
width = 8
height = width / 1.618

fontsize = 27
legend_font_size = 23
# fontsize = 18
# legend_font_size = 12
matplotlib.rcParams.update(
    {
        "font.size": fontsize,
        "figure.figsize": (width, height),
        "figure.facecolor": "white",
        "savefig.dpi": 72,
        "figure.subplot.bottom": 0.125,
        "figure.edgecolor": "white",
        "xtick.labelsize": fontsize,
        "ytick.labelsize": fontsize,
        "lines.linewidth": 3,
    }
)

@ticker.FuncFormatter
def hour_major_formatter(x, pos):
    x = float(f"{x/3600:.2f}")
    if x % 1 == 0:
        x = str(int(x))
    else:
        x = f"{x:.2f}"
    return x

In [ ]:
def moving_average(data_list, window_size=100):
    res_list = list()
    for i in range(len(data_list) - window_size):
            res_list.append(sum(data_list[i:i+window_size])/window_size)
    return res_list

## Best configuration(s)

In [ ]:
output_best_configuration(results_path, "", k=3)

## Information about evaluations

In [ ]:
display(Markdown(f"{len(results_df)} evaluations"))
display(Markdown(f"{len(np.unique(results_df.arch_seq))} unique architectures"))

## Search trajectory


### Scatter Plot

In [ ]:
plt.figure()

plt.scatter(results_df.elapsed_sec, results_df.objective)

plt.ylabel("Objective")
plt.xlabel("Time (hour)")

plt.xlim(0)

ax = plt.gca()
ax.xaxis.set_major_formatter(hour_major_formatter)

plt.grid()
plt.show()

## Balsam Workload
### Profile

In [ ]:
times = [to_sec(t)-start_timestamp for t in balsam_workload['timestamps']]
num_running = (np.array(balsam_workload['num_jobs_running']) - 1)
try:
    nworkers = init_infos['num_workers']
except:
    nworkers = int(input('How many workers do you have ?'))
num_running_norm = num_running / nworkers

plt.fill_between(times, num_running_norm, step="post", label='real')
plt.plot(moving_average(times), moving_average(num_running_norm), 'r:', linewidth=3, label='avrg')

plt.xlabel('Time (s)')
plt.ylabel('Node utilization')
plt.xlim(0)
plt.ylim(0)
plt.legend()

ax = plt.gca()
ax.xaxis.set_major_formatter(hour_major_formatter)

plt.grid()
plt.show()

### Pie chart

In [ ]:
def integrate_square(x, y, offset=0):
    res = 0
    for i in range(len(x)-1):
        res += (x[i+1] - x[i]) * y[i]
    return res

nworkers = init_infos['num_workers']
total_available = nworkers * times[-1]
used_time = integrate_square(times, num_running)
unused_time = total_available - used_time
perc_used_time = int(used_time / total_available * 100)
perc_unused_time = int(unused_time / total_available * 100)
label_used_time = f'{perc_used_time}% used'
label_unused_time = f'{perc_unused_time}% unused'
lpie = plt.pie([used_time, unused_time],
               labels=[label_used_time, label_unused_time],
               colors=['green', 'red'])
plt.show()